In [1]:
import pandas as pd
import re
import os
import math
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy import optimize
from scipy import stats
import sympy as sy
from scipy.optimize import brentq as root
import signal
%matplotlib inline

In [2]:
def normalize_column(df2,reads,channels,string):
    df1=df2[df2['Search ID'] == string][channels]
    df1.columns=reads
    df1=df1.reset_index(drop=True)
    temp1=list(df1['37'].values)
    for i in reads:
        temp=df1[i].values/temp1
        df1[i]=temp
    Seq=df2[df2['Search ID'] == string][["Sequence", "Protein Group Accessions"]]
    Seq=Seq.reset_index(drop=True)
    return df1,Seq

def fctSigmoidTR0(x,Pl,a,b):
    return (1 - Pl)*1/(1+np.exp(-(a/x-b))) + Pl

def fctSigmoidTR1(x,Pl,a,b):
    return -((1 - Pl) * (np.exp(-(a/x - b)) * (a/x**2))/(1 + np.exp(-(a/x - b)))**2)

def fctSigmoidTR2(x,Pl,a,b):
    return -((1 - Pl) * 1 * (np.exp(-(a/x - b)) * (a/x**2) * (a/x**2) - np.exp(-(a/x - b)) * (a * (2 * x)/(x**2)**2))/(1 + np.exp(-(a/x - b)))**2 - (1 - Pl) * 1 * (np.exp(-(a/x - b)) * (a/x**2)) *(2 * (np.exp(-(a/x - b)) * (a/x**2) * (1 + np.exp(-(a/x - b)))))/((1 + np.exp(-(a/x - b)))**2)**2)


startPars=[0, 550, 10]

def rSquared(y,z):
    y=np.array(y)
    z=np.array(z)
    ssTot = np.nansum((y - np.nanmean(y))**2)
    ssRes = np.nansum((y - z)**2)
    r2 = 1 - (ssRes/ssTot)
    return r2

def fitSigmoidTR(xVec, yVec, startPars, maxAttempts, fixT0=True, method=None):
    varyPars = 0
    attempts = 0
    repeatLoop = True
    validValues = []
    m=1234
    for i in yVec:
        if math.isnan(float(i)):
            validValues.append(0)
        else:
            validValues.append(1)
    if sum(validValues) <=2:
        m = [float('nan'),float('nan')]
        r_squared=0
    else:
        yVec=[yVec[i] for i in range(0,len(yVec)) if validValues[i] == 1]
        xVec=[xVec[i] for i in range(0,len(xVec)) if validValues[i] == 1]
        while (repeatLoop & (attempts < maxAttempts)):
            temp=(1 + varyPars*(np.random.uniform(-0.2, 0.2,(1,))[0]))
            parTmp = [i*temp for i in startPars]
            try:
                m=optimize.curve_fit(fctSigmoidTR0,xVec,yVec, parTmp, check_finite=False, bounds=([0.0,1e-5,1e-5], [1.5, 15000, 250]), method=method)
                attempts = attempts + 1
                varyPars = 1
                if not(all(np.isnan(m[0]))):
                    repeatLoop = False
            except RuntimeError,ValueError:
                m = [float('nan'),float('nan')]
                r_squared=0
        residuals = yVec- fctSigmoidTR0(xVec, m[0][0],m[0][1],m[0][2])
        ss_res = np.nansum(residuals**2)
        ss_tot = np.nansum((yVec-np.nanmean(xVec))**2)
        r_squared = 1 - (ss_res / ss_tot)
    return m[0],r_squared

def plot_fit(df,joint, xVec,startPars,title,plot=False):
    anorm=df.ix[joint]
    anorm = anorm[(((anorm.iloc[0:,6] < 0.6) & (anorm.iloc[0:,6] > 0.4)) & ((anorm.iloc[0:,8] < 0.3) & (anorm.iloc[0:,9] < 0.2)))]
    a=anorm.median(0)
    afitmodel,rsquared=fitSigmoidTR(xVec, a, startPars, 500, fixT0=True)
    afit=[]
    for i in temps:
        afit.append(fctSigmoidTR0(i,afitmodel[0],afitmodel[1],afitmodel[2]))
    r2=rSquared(a,afit)
    if plot == True:
        plt.scatter(xVec,a)
        plt.plot(xVec,afit)
        plt.xlim(35,66)
        plt.xlabel('Temperature')
        plt.ylabel('Fold Change')
        plt.title(title+'\n'+'R-Squared (noSUM): '+str(rsquared)+'\n'+'R-Squared (SUM): '+str(r2))
        plt.savefig('Figures/'+title+ ' Fit Plot.png')
        plt.show
    return a.tolist(),afit,afitmodel,rsquared,r2

def scale_factor(val1,fit1,r1,val2,fit2,r2):
    if r1 > r2:
        normcurve = fit1
    else:
        normcurve = fit2

    val1coeff=[normcurve[i]/val1[i] for i in range(0,len(val1))]
    val2coeff=[normcurve[i]/val2[i] for i in range(0,len(val2))]
    return val1coeff,val2coeff

def meltingPoint(model,xRange):
    try:
        if (model[1] == 0):
            r=float('nan')
        else:
            Pl=model[0][0]
            a=model[0][1]
            b=model[0][2]
            def calc(i):
                return fctSigmoidTR0(i,Pl,a,b)-0.5
            r=root(calc,min(xRange),max(xRange))
    except ValueError:
        r=float('nan')
    return r

def inflectionPoint(model,xRange):
    try:
        if (model[1] == 0):
            r=float('nan')
        else:
            Pl=model[0][0]
            a=model[0][1]
            b=model[0][2]
            def calc(i):
                return fctSigmoidTR2(i,Pl,a,b)
            r=root(calc,min(xRange),max(xRange))
    except ValueError:
        r=float('nan')
    return r

def meltingCurveSlope(model, xInfl):
    try:
        if (model[1] == 0):
            r=float('nan')
        else:
            Pl=model[0][0]
            a=model[0][1]
            b=model[0][2]
            r=fctSigmoidTR1(xInfl,Pl,a,b)
    except ValueError:
        r=float('nan')
    return r

def timeout(signum, frame):
#     print('TimeOut, changing method')
    raise Exception('changing method')

def fitting(df1,temps,startPars,title,df_info):
    df_fit = pd.DataFrame(columns=df1.columns,index=range(0,len(df1.index)))
    df_fit.columns=[title+'_'+str(df1.columns[i]) for i in range(0,len(df1.columns))]
    df_param=pd.DataFrame(columns=range(0,3),index=range(0,len(df1.index)))
    df_param.columns=[title+'_'+i for i in ['Pl','a','b']]
    df_R=[float('nan')]*len(df1.index)
    df_min=[float('nan')]*len(df1.index)
    df_infl=[float('nan')]*len(df1.index)
    df_slope=[float('nan')]*len(df1.index)
    colnames=[title+'_'+i for i in ['R','min','infl','slope']]
    for i in range(0,len(df1.index)):
#     for i in range(0,25):
        if i%1000==0:
            print(title+'_'+str(i))
        pts = df1.ix[i].tolist()
        validValues=[]
        for k in pts:
            if math.isnan(float(k)):
                validValues.append(0)
            else:
                validValues.append(1)
        pts1=pts
        pts=[pts[k] for k in range(0,len(pts)) if validValues[k] == 1]
        temps1=[temps[k] for k in range(0,len(temps)) if validValues[k] == 1]
        
        if sum(validValues) <= 2:
            continue
        else:
            signal.signal(signal.SIGALRM, timeout)
            signal.alarm(2)
            try:
                dffitmodel = fitSigmoidTR(temps1, pts, startPars, 500, fixT0=True)
            except Exception:
                signal.signal(signal.SIGALRM, timeout)
                signal.alarm(2)
                try:
                    dffitmodel = fitSigmoidTR(temps1, pts, startPars, 500, fixT0=True, method='dogbox')
                except Exception:
                    signal.signal(signal.SIGALRM, timeout)
                    signal.alarm(2)
                    try:
                        dffitmodel = fitSigmoidTR(temps1, pts, startPars, 500, fixT0=True, method='trf')
                    except Exception:
                        print('Notfound '+title+'_'+str(i))
                        dffitmodel = [float('nan'),float('nan')]
            if not(np.isnan(dffitmodel[1])):
                df_param.ix[i]=list(dffitmodel[0])
                dffit=[float('nan')] * len(temps)
                for j in range(0,len(temps)):
                    if validValues[j] == 1:
                        dffit[j]=fctSigmoidTR0(temps[j],dffitmodel[0][0],dffitmodel[0][1],dffitmodel[0][2])
                df_fit.ix[i]=dffit
                dffit=[dffit[k] for k in range(0,len(dffit)) if validValues[k] == 1]
                slope1, intercept1, r_value1, p_value1, std_err1 = sp.stats.linregress(pts,dffit)
                df_R[i]=r_value1**2
#                 df_R[i]=rSquared(pts1,df_fit.ix[i].tolist())
                df_min[i]=meltingPoint(dffitmodel,temps1)
                df_infl[i]=inflectionPoint(dffitmodel,temps1)
                df_slope[i]=meltingCurveSlope(dffitmodel, df_infl[i])
            else:
                continue
    df_res=pd.concat([df_fit,df_param],1)
    df_res[colnames[0]]=df_R
    df_res[colnames[1]]=df_min
    df_res[colnames[2]]=df_infl
    df_res[colnames[3]]=df_slope
    df_info=df_info.reset_index(drop=True)
    df_res=pd.concat([df_info,df_res],1)
    df_res.to_csv('../Result_trip/'+title+'_'+'fitting.csv',index=False)
    return df_fit,df_param,df_R,df_min,df_infl,df_slope

In [3]:
#INIT
data=pd.read_csv('combined.txt',sep='\t',low_memory=False)
mycols=range(0,16)
dataReduced=data[data.columns[mycols]]
ctrl1Name='E'
treated1Name='G'
ctrl2Name='F'
treated2Name='H'
ctrl3Name='I'
treated3Name='J'
reads = ["37", "40", "43", "46", "49", "52", "55", "58", "61", "64"]
channels = ["126", "127N", "127C", "128N", "128C", "129N", "129C", "130N", "130C", "131"]

In [4]:
ctrl1,ctrl1seq=normalize_column(dataReduced,reads,channels,ctrl1Name)
treated1,treated1seq=normalize_column(dataReduced,reads,channels,treated1Name)
ctrl2,ctrl2seq=normalize_column(dataReduced,reads,channels,ctrl2Name)
treated2,treated2seq=normalize_column(dataReduced,reads,channels,treated2Name)
ctrl3,ctrl3seq=normalize_column(dataReduced,reads,channels,ctrl3Name)
treated3,treated3seq=normalize_column(dataReduced,reads,channels,treated3Name)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:7: RuntimeWarning: invalid value encountered in divide
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:7: RuntimeWarning: divide by zero encountered in divide


In [5]:
Pgroups=dataReduced.drop_duplicates('Protein Group Accessions')['Protein Group Accessions']
Protein_desc=dataReduced.drop_duplicates('Protein Group Accessions')['Protein Descritions']
Desc=Protein_desc.tolist()
gene_name=dataReduced['Gene.Names'].tolist()
Pgroups=pd.DataFrame([Pgroups.tolist(),Desc,gene_name]).T
Pgroups.columns=["Protein Group Accessions", "Protein_Description", "Gene_Name"]

In [6]:
Seqs=dataReduced.drop_duplicates('Sequence')[['Unique Sequence ID','Sequence',"Protein Group Accessions"]]
Seqs.columns=["Unique_Sequence_ID", "Sequence", "Protein_Group_Accessions"]

In [9]:
temps=[37,40,43,46,49,52,55,58,61,64]
ctrl1Read=list(ctrl1.index)
treated1Read=list(treated1.index)
ctrl2Read=list(ctrl2.index)
treated2Read=list(treated2.index)
ctrl3Read=list(ctrl2.index)
treated3Read=list(treated2.index)

In [10]:
joint1=list(set(ctrl1Read).intersection(treated1Read))
joint2=list(set(ctrl2Read).intersection(treated2Read))
joint3=list(set(ctrl3Read).intersection(treated3Read))

In [11]:
a,afit,afitmodel,rsquared,ar=plot_fit(ctrl1,joint1,temps,startPars,'ctrl1')
b,bfit,bfitmodel,rsquared,br=plot_fit(treated1,joint1,temps,startPars,'treated1')
c,cfit,cfitmodel,rsquared,cr=plot_fit(ctrl2,joint2,temps,startPars,'ctrl2')
d,dfit,dfitmodel,rsquared,dr=plot_fit(treated2,joint2,temps,startPars,'treated2')
e,efit,efitmodel,rsquared,er=plot_fit(ctrl3,joint3,temps,startPars,'ctrl3')
f,ffit,ffitmodel,rsquared,fr=plot_fit(treated3,joint3,temps,startPars,'treated3')

In [12]:
acoeff,bcoeff=scale_factor(a,afit,ar,b,bfit,br)
ccoeff,dcoeff=scale_factor(c,cfit,cr,d,dfit,dr)
ecoeff,fcoeff=scale_factor(e,efit,er,f,ffit,fr)
allcoeff=pd.DataFrame([acoeff,bcoeff,ccoeff,dcoeff,ecoeff,fcoeff],columns=reads,index=['a.coeff','b.coeff','c.coeff','d.coeff','e.coeff','f.coeff'])

In [13]:
ctrl1norm=ctrl1*acoeff
ctrl2norm=ctrl2*ccoeff
treated1norm=treated1*bcoeff
treated2norm=treated2*dcoeff
ctrl3norm=ctrl3*ecoeff
treated3norm=treated3*fcoeff

In [14]:
ctrl1Unorm=pd.concat([ctrl1seq,ctrl1norm], 1).groupby('Sequence').median()
treated1Unorm=pd.concat([treated1seq,treated1norm], 1).groupby('Sequence').median()
ctrl2Unorm=pd.concat([ctrl2seq,ctrl2norm], 1).groupby('Sequence').median()
treated2Unorm=pd.concat([treated2seq,treated2norm], 1).groupby('Sequence').median()
ctrl3Unorm=pd.concat([ctrl3seq,ctrl3norm], 1).groupby('Sequence').median()
treated3Unorm=pd.concat([treated3seq,treated3norm], 1).groupby('Sequence').median()
combinedUnorm=pd.concat([ctrl1Unorm,treated1Unorm,ctrl2Unorm,treated2Unorm,ctrl3Unorm,treated3Unorm],1)

In [15]:
UniProt_Accession = set(combinedUnorm.index).intersection(Seqs.Sequence)
Seqs=Seqs[Seqs.Sequence.isin(UniProt_Accession)]
Seqs=Seqs.sort_values('Sequence')
Seqs=Seqs.reset_index(drop=True)

In [16]:
ctrl1norm=combinedUnorm.iloc[:,0:10]
treated1norm=combinedUnorm.iloc[:,10:20]
ctrl2norm=combinedUnorm.iloc[:,20:30]
treated2norm=combinedUnorm.iloc[:,30:40]
ctrl3norm=combinedUnorm.iloc[:,40:50]
treated3norm=combinedUnorm.iloc[:,50:60]

In [17]:
ctrl1_fit,ctrl1_param,ctrl1_R,ctrl1_min,ctrl1_infl,ctrl1_slope=fitting(ctrl1norm,temps,startPars,'ctrl1',Seqs)
treated1_fit,treated1_param,treated1_R,treated1_min,treated1_infl,treated1_slope=fitting(treated1norm,temps,startPars,'treated1',Seqs)
ctrl2_fit,ctrl2_param,ctrl2_R,ctrl2_min,ctrl2_infl,ctrl2_slope=fitting(ctrl2norm,temps,startPars,'ctrl2',Seqs)
treated2_fit,treated2_param,treated2_R,treated2_min,treated2_infl,treated2_slope=fitting(treated2norm,temps,startPars,'treated2',Seqs)
ctrl3_fit,ctrl3_param,ctrl3_R,ctrl3_min,ctrl3_infl,ctrl3_slope=fitting(ctrl3norm,temps,startPars,'ctrl3',Seqs)
treated3_fit,treated3_param,treated3_R,treated3_min,treated3_infl,treated3_slope=fitting(treated3norm,temps,startPars,'treated3',Seqs)

ctrl1_0
Notfound ctrl1_81
Notfound ctrl1_407
ctrl1_1000


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:20: RuntimeWarning: overflow encountered in double_scalars


Notfound ctrl1_1333
Notfound ctrl1_1770
ctrl1_2000
ctrl1_3000
ctrl1_4000
Notfound ctrl1_4381
ctrl1_5000
ctrl1_6000
Notfound ctrl1_6270
Notfound ctrl1_6426
ctrl1_7000
Notfound ctrl1_7528
ctrl1_8000
ctrl1_9000
Notfound ctrl1_9601
Notfound ctrl1_9602
Notfound ctrl1_9693
ctrl1_10000
Notfound ctrl1_10328
Notfound ctrl1_10812
ctrl1_11000
Notfound ctrl1_11121
Notfound ctrl1_11227
ctrl1_12000
Notfound ctrl1_12429
Notfound ctrl1_12708
Notfound ctrl1_12745
ctrl1_13000
Notfound ctrl1_13642
ctrl1_14000
Notfound ctrl1_14253
Notfound ctrl1_14300
ctrl1_15000
ctrl1_16000
Notfound ctrl1_16986
ctrl1_17000
Notfound ctrl1_17686
ctrl1_18000
Notfound ctrl1_18744
Notfound ctrl1_18833
ctrl1_19000
Notfound ctrl1_19092
Notfound ctrl1_19942
ctrl1_20000
ctrl1_21000
Notfound ctrl1_21078
treated1_0
treated1_1000
treated1_2000
Notfound treated1_2488
treated1_3000
treated1_4000
Notfound treated1_4690
treated1_5000
treated1_6000
Notfound treated1_6426
Notfound treated1_6476
Notfound treated1_6911
treated1_7000
treated

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:20: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:17: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:17: RuntimeWarning: invalid value encountered in double_scalars


ctrl3_10000
Notfound ctrl3_10377
Notfound ctrl3_10873
ctrl3_11000
ctrl3_12000
Notfound ctrl3_12396
Notfound ctrl3_12409
ctrl3_13000
Notfound ctrl3_13040
Notfound ctrl3_13180
Notfound ctrl3_13384
ctrl3_14000
Notfound ctrl3_14190
Notfound ctrl3_14191
ctrl3_15000
Notfound ctrl3_15292
Notfound ctrl3_15361
ctrl3_16000
Notfound ctrl3_16065
Notfound ctrl3_16278
ctrl3_17000
Notfound ctrl3_17686
ctrl3_18000
Notfound ctrl3_18932
ctrl3_19000
Notfound ctrl3_19884
Notfound ctrl3_19942
ctrl3_20000
Notfound ctrl3_20216
ctrl3_21000
Notfound ctrl3_21078
treated3_0
Notfound treated3_2
Notfound treated3_90
Notfound treated3_143
treated3_1000
treated3_2000
treated3_3000
Notfound treated3_3295
Notfound treated3_3388
treated3_4000
treated3_5000
Notfound treated3_5016
treated3_6000
treated3_7000
treated3_8000
treated3_9000
treated3_10000
Notfound treated3_10119
Notfound treated3_10508
Notfound treated3_10991
treated3_11000
Notfound treated3_11884
treated3_12000
Notfound treated3_12106
Notfound treated3_12738